# Langchain Tools


In [1]:
!pip install callbacks langchain_pinecone

  Preparing metadata (setup.py) ... one
  Using cached pinecone-5.4.2-py3-none-any.whl.metadata (19 kB)
  Using cached pytest_socket-0.7.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached pinecone_plugin_inference-3.1.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl.metadata (1.2 kB)
  Using cached pluggy-1.5.0-py3-none-any.whl.metadata (4.8 kB)
Using cached pinecone-5.4.2-py3-none-any.whl (427 kB)
Using cached pinecone_plugin_inference-3.1.0-py3-none-any.whl (87 kB)
Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl (6.2 kB)
Using cached pytest_socket-0.7.0-py3-none-any.whl (6.8 kB)
Using cached pluggy-1.5.0-py3-none-any.whl (20 kB)
  Created wheel for callbacks: filename=callbacks-0.3.0-py3-none-any.whl size=5689 sha256=90ac19b119e525560ac371c8bef139eb111b3dca7dd4a9eee2fb7043ef2deccc
  Stored in directory: /Users/katie/Library/Caches/pip/wheels/b9/d3/16/a3f93d3628ff8ca6025ef62cea56ca0e49d0be67cd70923e4f
Successfully buil

# LangChain

In [9]:
from typing import Union, List
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain_openai import ChatOpenAI
from langchain.schema import AgentAction, AgentFinish
from langchain.tools import Tool, tool
from langchain.tools.render import render_text_description
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import LLMResult
from callbacks import AgentCallbackhandler
load_dotenv()

class AgentCallbackhandler(BaseCallbackHandler):
    def on_llm_start(
            self, serialized:Dict[str,Any],prompts:List[str],**kwargs:Any
    )->Any:
        """Run when LLM starts running."""
        print(f'**Prmpt to LLM was:**\n{prompts[0]}')
        print('***')
    def on_llm_end(self, response:LLMResult, **kwargs:Any)->Any:
        """Run when LLM ends running."""
        print('***LLM Response:***\n{response.generations[0][0].text}')
        print('****')
    
@tool
def get_text_length(text:str)->int:
    """Returns the length of a text by characters"""
    text=text.strip("'\n").strip(
        '"'
    )
    return len(text)

def find_tool_by_name(tools:List[Tool],tool_name:str)->Tool:
    for tool in tools:
        if tool.name == tool.name:
            return tool
    raise ValueError(f'Tool with name {tool_name} not found')

tools = [get_text_length]
template="""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:
"""

prompt = PromptTemplate.from_template(template=template).partial(
    tools=render_text_description(tools), 
    tool_names=', '.join([t.name for t in tools])
)
llm=ChatOpenAI(temperature=0,stop='Observation', callbacks=[AgentCallbackhandler()])
intermediate_steps=[]
agent = {'input':lambda x:x['input']}| prompt | llm | ReActSingleInputOutputParser()
agent_step:Union[AgentAction,AgentFinish] = agent.invoke({
    "input":"What is the length in characters of the text DOG?",
    "agent_scrachpad":intermediate_steps
})
print(agent_step)
if isinstance(agent_step, AgentAction):
    tool_name=agent_step.tool
    tool_to_use=find_tool_by_name(tools,tool_name)
    tool_input=agent_step.tool_input
    observation=tool_to_use.func(str(tool_input))
    print(f"{observation=}")
    intermediate_steps.append((agent_step, str(observation)))

if isinstance(agent_step, AgentFinish):
    print(agent_step.return_values)


ImportError: cannot import name 'AgentCallbackhandler' from 'callbacks' (/opt/anaconda3/envs/llms/lib/python3.11/site-packages/callbacks/__init__.py)

# Agent LangChain + Tavily to find account

In [7]:
import os
from langchain import hub
from langchain.agents import (
    create_react_agent,
    AgentExecutor,
)
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
def lookup(name: str) -> str:
    llm = ChatOpenAI(
        temperature=0,
        model_name='gpt-4o-mini',
        openai_api_key=os.environ['OPENAI_API_KEY'],
    )
    # llm = ChatOllama(model='llama3')
    template = """given the name {name_of_person} I want you to find a link to their Twitter profile page, and extract from it their username
    In Your Final answer only the person:s username which is extracted from: https://x.com/USERNAME"""
    prompt_template = PromptTemplate(
        template=template, input_variables=[name_of_person']
    )
    tools_for_agent = [
        Tool(
            name='Crawl Google 4 Twitter profile page',
            func=get_profile_url_tavily,
            description='useful for when you need get the Twitter Page URL',
        )
    ]
    react_prompt = hub.pull('hwchase17/react')
    agent = create_react_agent(llm=llm, tools=tools_for_agent, prompt=react_prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools_for_agent, verbose=True)
    result = agent_executor.invoke(
        input={'input': prompt_template.format_prompt(name_of_person=name)}
    )
    return result['output']


ModuleNotFoundError: No module named 'langchain_ollama'

# QA LangChain vectorstore

In [6]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.runnables import RunnablePassthrough
load_dotenv()
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)
embeddings=OpenAIEmbeddings()
llm = ChatOpenAI()
query = 'What is Pinecone in machine learning?'
chain = PromptTemplate.from_template(template=query) | llm
vectorstore=PineconeVectorStore(
    index_name=os.environ['INDEX_NAME'],embedding=embeddings
)
retrieval_qa_chat_prompt=hub.pull('langchain-ai/retrieval-qa-chat')
combine_docs_chain=create_stuff_documents_chain(llm,retrieval_qa_chat_prompt)
retrieval_chain = create_retrieval_chain(
    retriever=vectorstore.as_retriever(), combine_docs_chain=combine_docs_chain
)
result = retrieval_chain.invoke(input={'input':query})

template="""Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't now, don't try to make up an answer.
Use three sentenses maximum and keep the answer as concse as possible.
Always say 'thanks for asking!' at the end of the answer.

{context}

Question:{question}

Helpful Answer: 
"""
custom_rag_prompt = PromptTemplate.from_template(template)
rag_chain = (
    {'context':vectorstore.as_retriever()| format_docs, 'question':RunnablePassthrough()}
    |custom_rag_prompt
    |llm
)
res=rag_chain.invoke(query)
print(res.content)

Pinecone is a fully managed cloud-based vector database designed for fast and scalable retrieval of similar data points based on their vector representations, supporting large-scale ML applications with high query throughput and low latency search. Pinecone is secure, user-friendly, and provides real-time updates for efficient database maintenance.
Thanks for asking!


# QA LangChain PDF

In [ ]:
import os
import pdfplumber
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from PyPDF2 import PdfReader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain_community.vectorstores import FAISS
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import hub
load_dotenv()
# os.environ['OPEN_AI_KEY']
from langchain.docstore.document import Document
loader = PyPDFLoader('/Users/katie/Documents/development/python/llm/vectorstor-in-memory/ocred_doc1.pdf')
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30,separator='\n')
docs=text_splitter.split_documents(documents=documents)
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs,embeddings)
vectorstore.save_local('/Users/katie/Documents/development/python/llm/vectorstor-in-memory/faiss_index_react')
new_vectorstore=FAISS.load_local('/Users/katie/Documents/development/python/llm/vectorstor-in-memory/faiss_index_react', embeddings, allow_dangerous_deserialization=True)
retrieval_qa_chat_prompt = hub.pull('langchain-ai/retrieval-qa-chat')
combine_docs_chain = create_stuff_documents_chain(OpenAI(), retrieval_qa_chat_prompt)
retrieval_chain= create_retrieval_chain(new_vectorstore.as_retriever(),combine_docs_chain)
res = retrieval_chain.invoke({'input':'Give me the gist of ReAct in 3 sentences'})
print(res['answer'])


# Practice Langchain PromptTemplate 1

**input_variables:**
When partial_variables are not included, 1st variable have to be question.

In [17]:
import os
from langchain.llms import OpenAI
from langchain import PromptTemplate
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')   
openai = OpenAI()
template: str = """/
You are an AI agent. give responses to the following/ 
question: {question}. Use simple words and short sentence.
"""
prompt = PromptTemplate.from_template(template=template)
prompt_formatted_str: str = prompt.format(question="Who is founder of Apple?")
print(openai.predict(prompt_formatted_str))


The founder of Apple is Steve Jobs.


# Summarise Langchain

In [20]:
import os
from langchain.llms import OpenAI
from langchain import PromptTemplate
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')   
openai = OpenAI()
template: str = """
    Given the information about article {question} I want you to create:
    1. a short summary
    2. list up 5 important things
"""
prompt = PromptTemplate.from_template(template=template)
prompt_formatted_str: str = prompt.format(
    question= 'The 5 paid subscriptions I actually use in 2025 as a Staff Software Engineer Tools I use that are cheaper than Netflix Jacob Bennett Level Up Coding Jacob Bennett · Follow Published in Level Up Coding · 5 min read · Jan 8, 2025 11.7K 291 I care a lot about the tools I use. Especially when they aren’t free. Here’s what I’m paying for in 2025 to improve my performance and productivity as a software engineer. Please note: None of the links in this article are affiliate links. Cursor: the best AI programming tool Cursor feels like a bargain for the level-up it provides me. I’m an average programmer with a strong preference for back-end projects, but Cursor lets me work across the full stack with the clarity and quality of a much better engineer. A chat I had with Cursor regarding a copy-pasted code block Along with code completion, Cursor has the best chat of any code-aware chat tool I’ve used. Usually I’ll just highlight a code block that I’m looking at, press CMD + L, ask a question in natural language, and Cursor load all of the relevant context from the current project and external sources (e.g. package docs) and give me an immediately-useful answer. Learn more about Cursor → What about GitHub Copilot? I used Copilot for 8 months in 2024. It was great! (I even included it on my 2024 list.) But Cursor blows it out of the water. Copilot was the most accurate when I was writing generic/boilerplate code (e.g. API endpoints) or test cases with lots of repeated code blocks. Anything more complex (especially logic spanning multiple files and modules) and Copilot was unreliable. Cursor feels like an actual context-aware pair programmer with a decent understanding of the project I’m looking at. Kagi: a better search engine than Google I measure the effectiveness of searches by how long it takes me to find what I was actually looking for. By that measure, Google has been steadily getting worse. When I search for something on Kagi, the correct result is in the first 2 links 95% of the time. It’s in the top 5 links 99% of the time. That just doesn’t happen with Google, Bing, etc. The consistently great results page is further boosted by the search personalization… The 5 paid subscriptions I actually use in 2025 as a Staff Software Engineer Tools I use that are cheaper than Netflix Jacob Bennett Level Up Coding Jacob Bennett · Follow Published in Level Up Coding · 5 min read · Jan 8, 2025 11.7K 291 I care a lot about the tools I use. Especially when they aren’t free. Here’s what I’m paying for in 2025 to improve my performance and productivity as a software engineer. Please note: None of the links in this article are affiliate links. Cursor: the best AI programming tool Cursor feels like a bargain for the level-up it provides me. I’m an average programmer with a strong preference for back-end projects, but Cursor lets me work across the full stack with the clarity and quality of a much better engineer. A chat I had with Cursor regarding a copy-pasted code block Along with code completion, Cursor has the best chat of any code-aware chat tool I’ve used. Usually I’ll just highlight a code block that I’m looking at, press CMD + L, ask a question in natural language, and Cursor load all of the relevant context from the current project and external sources (e.g. package docs) and give me an immediately-useful answer. Learn more about Cursor → What about GitHub Copilot? I used Copilot for 8 months in 2024. It was great! (I even included it on my 2024 list.) But Cursor blows it out of the water. Copilot was the most accurate when I was writing generic/boilerplate code (e.g. API endpoints) or test cases with lots of repeated code blocks. Anything more complex (especially logic spanning multiple files and modules) and Copilot was unreliable. Cursor feels like an actual context-aware pair programmer with a decent understanding of the project I’m looking at. Kagi: a better search engine than Google I measure the effectiveness of searches by how long it takes me to find what I was actually looking for. By that measure, Google has been steadily getting worse. When I search for something on Kagi, the correct result is in the first 2 links 95% of the time. It’s in the top 5 links 99% of the time. That just doesn’t happen with Google, Bing, etc. The consistently great results page is further boosted by the search personalization…'
)
print(openai.predict(prompt_formatted_str))


Summary:
The article discusses the top 5 paid subscriptions that the author, a Staff Software Engineer, uses in 2025 to improve their performance and productivity. They highlight the benefits of each tool and why they find them worth the investment.

5 Important Things:

1. The author values the tools they use and believes that investing in them is important for their performance as a software engineer.
2. They use Cursor, an AI programming tool, for code completion and code-aware chat.
3. The author finds Cursor to be more reliable and useful than GitHub Copilot.
4. They also use Kagi, a search engine, which provides consistently accurate results and personalized search options.
5. The author emphasizes the importance of personalization and effectiveness when it comes to choosing tools for their work. 
